In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/Shared drives/數位醫學/Case Presentation 1/digital

Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/1403v7Kv9G42MNoj5JOfYxijc9VTDmStt/Case Presentation 1/digital


In [2]:
import re
import pandas as pd
import numpy as np
import csv
from os import rename
from os import listdir
PATH = './'
INPUT_PATH = PATH+'test_data/'
OUTPUT_PATH = PATH+'result/'

In [3]:
rx_dict = { 
    #小心不要加到空白格！！！
    
    #{smok|tobacco|cigarette}
    'y_1': re.compile(r'(?P<y_1>([Ss]mok|[Tt]obacco|[Cc]igarette)[_a-z]*)'),
     
    # {deny|not|no} +{smok|tobacco|cigarette}
    'no_1': re.compile(r'(?P<no_1>([Dd]en[ies|y]|not[ ]|n\'t[ ])(\w|\s)*([Ss]mok|[Tt]obacco|[Cc]igarette))'),
    'no_2': re.compile(r'(?P<no_6>[Nn]o[ ]([Ss]mok|[Tt]obacco|[Cc]igarette))'),
    
    # {quit|former|prior} +{smok|tobacco|cigarette}
    'p_1': re.compile(r'(?P<p_1>([Qq]uit|[Ff]ormer|[Pp]rior)(\w|\s)*([Ss]mok|[Tt]obacco|[Cc]igarette))'),
    'p_2': re.compile(r'(?P<p_10>[Ss]mok(\w|\s)*past)'),
    
    # Label:
    'Label': re.compile(r'(?P<Label>(Tobacco|Smok(e|ing))(\s)*[\:])'),
    
}

rx_w_label_dict ={
    'no_101': re.compile(r'(?P<no_52>[Nn]o[t]?\s)'),
    'no_102': re.compile(r'(?P<no_51>[Dd]en[y|ies](\w|\s)*)'),
    'p_101': re.compile(r'(?P<no_51>[Qq]uit(\w|\s)*)'),
}

In [4]:
# Look for regx
def _parse_line(line):
    return_key=None
    return_match=None
    for key, rx in rx_dict.items():
        match = rx.search(line)
        if match:
            return_key=key
            return_match=match.group(1)
    return return_key,return_match

In [5]:
# Look for regx
# Have the same usage with _parse_line()
# use to identify pattern of form "Tobacco:\n Quit..."(found in training_data＿68)
def Deal_Label_line(line):
    return_key=None
    return_match=None
    for key, rx in rx_w_label_dict.items():
      match2 = rx.search(line)
      if (match2):
        return_key=key
        return_match=match2.group(1)
    return return_key,return_match

In [6]:
# Read test files from INPUT_PATH
# Identify pattern
# Ouput: test_o.csv
# Form : a table with col: [ID|pattern|plain_text|type] (plain_text contains only one line)
def parse_file():
    files = listdir(INPUT_PATH)
    sorted_files =  sorted(files)
    data=[]
    for file_ in (sorted_files[1:]):
        f = open(INPUT_PATH + file_,"r",encoding="utf-8", errors='ignore')
        row = {
            'ID'       :None,
            'pattern'  :None,
            'plain_txt':None,
            'type'     :None,
        }
        num = (file_[5:7])
        row['ID']=num

        line = f.readline()
        while line:
            
            key,match = _parse_line(line)
            if(match):
                row['pattern']=match
                row['plain_txt']=line
                if(key[0]=='y'):
                    row['type']=0
                elif(key[0]=='n'):
                    row['type']=1
                elif(key[0]=='p'):
                    row['type']=2
                elif(key[0]=='L'):
                    line = f.readline()
                    key2,match2 = Deal_Label_line(line)
                    if(match2):
                        row['pattern']=match+match2
                        row['plain_txt']=match+line
                        if(key2[0]=='n'):
                            row['type']=1
                        elif(key2[0]=='p'):
                            row['type']=2
                        else:
                            row['type']=0
                break
            else:
                row['type']=3
            line = f.readline()
       
        data.append(row)
        #print(num)
        f.close()
    df = pd.DataFrame(data)
    df.to_csv(OUTPUT_PATH+'test_o.csv', index=False) 

In [7]:
# Read  ans.csv    ->  list_ans[] 
#       test_o.csv ->  list_test[]
# Compare ans_type(工人智慧><) with test_type(our output) and merge diff_rows into one table
# Ouput the table in html form

def judge_output():
    # ans.csv->list_ans[]
    list_ans_type=[]
    with open(OUTPUT_PATH+'ans.csv', newline='') as csvfile:
        rows = csv.DictReader(csvfile)
        for row in rows:
            list_ans_type.append(row['label'])
    csvfile.close()
    
    #test_o.csv->list_test[]
    list_test_type=[]
    list_test_plain=[]
    list_test_pattern=[]
    with open(OUTPUT_PATH+'test_o.csv', newline='') as csvfile:
        rows = csv.DictReader(csvfile)
        for row in rows:
            list_test_type.append(row['type'])
            list_test_plain.append(row['plain_txt'])
            list_test_pattern.append(row['pattern'])
    csvfile.close()
    
    #calcaulate score and print out the diff part
    diff=[]
    score=100.0
    penalty=(100/40)
    for i in range(40):
        row = {
            'ID'       :None,
            'plain_txt':None,
            'pattern'  :None,
            'ans_type' :None,
            'diff_type':None,
        }
        
        if(list_ans_type[i]!=list_test_type[i]):
            row['ID']       =i+1
            row['ans_type'] =(list_ans_type[i])
            row['diff_type']=(list_test_type[i])
            row['plain_txt']=(list_test_plain[i])
            row['pattern']  =(list_test_pattern[i])        
            score-=penalty
            diff.append(row)
            
        df = pd.DataFrame(diff)
  
    print ("Your Score is: %f" % (score) )

    #df.to_csv(PATH+'/result/'+'diff_o.csv', index=False)
    html = df.to_html()
    text_file = open(OUTPUT_PATH +'diff_rows.html', "w")
    text_file.write(html)
    text_file.close()

In [8]:
def convert_to_inputform(str):
    # ans.csv->list_ans[]
    data=[]
    data_txt=[]
    data_label=[]
    with open(OUTPUT_PATH+'test_o.csv', newline='') as csvfile:
        rows = csv.DictReader(csvfile)
        for row in rows:
             if(row['plain_txt']==""):
                data_txt.append("unknown")
             else: 
                data_txt.append(row['plain_txt'])
             data_label.append(row['type'])
    csvfile.close()
    
    if(str=='train_form'):
        for i in range(41,80):
            row = {
                'NULL'   :None,
                'plain_txt'    :None,
                'type'  :None,
            }
            row['plain_txt']=(data_txt[i])
            row['type']=(data_label[i])
            data.append(row)
    elif(str=='test_form'):
        for i in range(40):
            row = {
                'NULL'   :None,
                'plain_txt'    :None,
                'type'  :None,
            }
            row['plain_txt']=(data_txt[i])
            row['type']=(data_label[i])
            data.append(row)        
    else:
        print('input_error')
    
    df = pd.DataFrame(data)
    df.to_csv(OUTPUT_PATH+str+'.csv', index=False)  

In [9]:
parse_file()
convert_to_inputform('test_form')
convert_to_inputform('train_form')
judge_output()

Your Score is: 82.500000
